In [2]:
import torch
import numpy as np
from torch.utils import data

# data/model/loss/optimize/train/eval

# 1 data

In [6]:
a = torch.Tensor([2,3,4])

In [14]:
torch.argmax?

In [15]:
torch.topk?

In [16]:
import torchvision

In [17]:
torchvision.datasets?

In [18]:
mnist_train = torchvision.datasets.FashionMNIST('../data',train=True,download=True,
                                                transform = torchvision.transforms.ToTensor())

In [19]:
mnist_test = torchvision.datasets.FashionMNIST('../data',train=False,download=True,
                                                transform = torchvision.transforms.ToTensor())

In [20]:
train_loader = data.DataLoader?

In [56]:
train_loader = data.DataLoader(mnist_train,shuffle=True,batch_size=256)
test_loader = data.DataLoader(mnist_test,shuffle=False,batch_size=256)

# 2 model

In [23]:
net = torch.nn.Sequential(torch.nn.Flatten(),torch.nn.Linear(28*28,128),
                         torch.nn.ReLU(),
                         torch.nn.Linear(128,10))

## loss

In [24]:
loss_func = torch.nn.CrossEntropyLoss()

## optimize

In [25]:
optimize = torch.optim.AdamW?

In [26]:
optimize = torch.optim.AdamW(net.parameters(),lr=0.02)

## init

In [38]:
def init_weight(l):
    if isinstance(l,torch.nn.Linear):
        l.weight.data.normal_(mean=0,std=0.1)

In [39]:
net.apply(init_weight)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=128, bias=True)
  (2): ReLU()
  (3): Linear(in_features=128, out_features=10, bias=True)
)

# 3 train

In [42]:
def accuracy(y_pred,y,debug= False):
    if len(y_pred.shape)>1 and y_pred.shape[1] >1:
        y_pred = torch.argmax(y_pred,-1)
    if debug:
        print(y_pred)
    return sum(y_pred == y)

In [43]:
tst_p = torch.Tensor([[0.8,0.1,0.1],[0.2,0.2,0.6]])
tst_y = torch.Tensor([0,1])

In [45]:
accuracy(tst_p,tst_y,True)

tensor([0, 2])


tensor(1)

In [63]:
class accumulator():
    def __init__(self,n):
        self.data = [0.0]*n
        
    def add(self,*args):
        self.data = [a+float(b) for a,b in zip(self.data,args)]
        
    def __getitem__(self,idx):
        return self.data[idx]

In [71]:
def eval_acc(data_iter,net):
    if isinstance(net,torch.nn.Module):
        net.eval()
    metrics = accumulator(2)
    for x,y in data_iter:
        y_pred = net(x)
        metrics.add(accuracy(y_pred,y),y.numel())
    return metrics[0]/metrics[1]

In [72]:
def train_epoch(data_iter,net,loss,optimize):
    if isinstance(net,torch.nn.Module):
        net.train()
    metrics = accumulator(3)
    for x,y in data_iter:
        y_pred = net(x)
        loss_tmp = loss(y_pred,y)
        metrics.add(loss_tmp*len(y),accuracy(y_pred,y),y.numel())
        ##bp
        optimize.zero_grad()
        loss_tmp.backward()
        optimize.step()
    return metrics[0]/metrics[2],metrics[1]/metrics[2]

In [73]:
def train(train_iter,test_iter,net,loss,optimize,num_epoch = 10):
    for i in range(num_epoch):
        ##train_step 
        train_metrics = train_epoch(train_iter,net,loss,optimize)
        acc_test = eval_acc(test_iter,net)
        print('epoch:%d\ttrain-loss:%f\ttrain-acc:%f\ttest-acc:%f'%(i,train_metrics[0],train_metrics[1],acc_test))

In [74]:
train(train_iter=train_loader,test_iter=test_loader,net=net,loss= loss_func,optimize=optimize,num_epoch=10)

epoch:0	train-loss:0.418182	train-acc:0.851067	test-acc:0.838200
epoch:1	train-loss:0.389322	train-acc:0.858467	test-acc:0.841500
epoch:2	train-loss:0.372359	train-acc:0.865650	test-acc:0.838600
epoch:3	train-loss:0.361586	train-acc:0.867300	test-acc:0.852400
epoch:4	train-loss:0.350666	train-acc:0.871350	test-acc:0.849600
epoch:5	train-loss:0.343054	train-acc:0.875567	test-acc:0.852500
epoch:6	train-loss:0.340757	train-acc:0.875950	test-acc:0.859400
epoch:7	train-loss:0.334956	train-acc:0.876617	test-acc:0.843600
epoch:8	train-loss:0.333300	train-acc:0.878033	test-acc:0.849700
epoch:9	train-loss:0.329943	train-acc:0.879117	test-acc:0.860800


In [76]:
list(net.parameters())

[Parameter containing:
 tensor([[ 0.0266, -0.1251,  0.0082,  ...,  0.0142,  0.0755, -0.1089],
         [ 0.0161, -0.0544,  0.0448,  ..., -0.0313, -0.0067, -0.0028],
         [-0.0922, -0.0747,  0.0458,  ..., -0.0873,  0.1171, -0.4024],
         ...,
         [ 0.0272,  0.1183, -0.1392,  ..., -0.0741,  0.1250, -0.0768],
         [-0.0244,  0.0390,  0.0808,  ...,  0.0925,  0.0629,  0.0304],
         [ 0.0535,  0.0826, -0.0231,  ...,  0.0269,  0.0158, -0.0448]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0960, -0.1129,  0.0497, -0.1024,  0.5894,  0.8789, -0.0600, -0.0261,
         -0.1892, -0.0735, -0.0433, -0.2822, -0.0922, -0.0881, -0.3114, -0.0645,
         -0.0906, -0.0801, -0.0617,  1.0136, -0.0164, -0.1040, -0.2366, -0.0460,
         -0.0544,  2.5766,  0.0033,  2.1449,  1.5758, -0.0432, -0.0359, -0.0698,
          0.0553, -0.0118,  0.1946, -0.0801, -0.0636,  1.0120, -0.0636, -0.0855,
          0.8600, -0.0795, -0.0764,  0.0053, -0.0554, -0.0673, -0.0930, -0.0966

In [77]:
torch.nn.Linear??